Based on the rate of consumption emissions, territorial emissions, fossil fuel type and the global carbon project, estimate the predicted annual temperature of the Philippines in 2050.

NOTES:

1. On the spikes, point out the events that caused the spike like the 1998 PH drought, the World War 2 in 1944-1945, El Niño/La Nina phenomenons.
2. Add the number of typhoons that entered the Philippines.
3. Agricultural output and Annual GDP
4. Oil export and consumption
5. Population

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

In [2]:
rainfall_ph = pd.read_csv("../data/worldbank/pr_1901_2016_PHL.csv")
temperature_ph = pd.read_csv("../data/worldbank/tas_1901_2016_PHL.csv")
temperature_global_annual = pd.read_csv("../data/datahub.io/global-temp/data/annual_csv.csv")

consumption_emissions = pd.read_csv("../data/global-carbon-project/consumption-emissions.csv")
territorial_emissions = pd.read_csv("../data/global-carbon-project/territorial-emissions.csv")
fossil_fuel_type = pd.read_csv("../data/global-carbon-project/fossil-fuel-type.csv")
global_carbon_project = pd.read_csv("../data/global-carbon-project/global-carbon-budget.csv")
worldbank_data = pd.read_csv("../data/worldbank/worldbank_ph_fr1960.csv")

In [3]:
wb_data = worldbank_data[worldbank_data['Country Code'] == 'PHL']\
    .drop(columns=['Country Name', 'Country Code', 'Indicator Name'])\
    .pivot_table(columns='Indicator Code')\
    .reset_index()\
    .rename(columns={'index': 'Year'})\
    .iloc[:-1]

wb_data['Year'] = wb_data['Year'].astype(int)

In [4]:
wb_data['NY.GDP.MKTP.CD']

0     6.684569e+09
1     7.256967e+09
2     4.399828e+09
3     4.875310e+09
4     5.271405e+09
5     5.784399e+09
6     6.371459e+09
7     6.809134e+09
8     7.591603e+09
9     8.408230e+09
10    6.687205e+09
11    7.408306e+09
12    8.017469e+09
13    1.008289e+10
14    1.378114e+10
15    1.489397e+10
16    1.709756e+10
17    1.964811e+10
18    2.270616e+10
19    2.750217e+10
20    3.245054e+10
21    3.564642e+10
22    3.714016e+10
23    3.321218e+10
24    3.140849e+10
25    3.073434e+10
26    2.986834e+10
27    3.319593e+10
28    3.788544e+10
29    4.257518e+10
30    4.431159e+10
31    4.541756e+10
32    5.297634e+10
33    5.436808e+10
34    6.408446e+10
35    7.411999e+10
36    8.284814e+10
37    8.234426e+10
38    7.220703e+10
39    8.299515e+10
40    8.102630e+10
41    7.626207e+10
42    8.135761e+10
43    8.390821e+10
44    9.137124e+10
45    1.030716e+11
46    1.222107e+11
47    1.493599e+11
48    1.741951e+11
49    1.683346e+11
50    1.995908e+11
51    2.241431e+11
52    2.5009

In [ ]:
from_year = 1910
to_year = 2016

tmph_mean_per_year = temperature_ph[(temperature_ph['year'] >= from_year) & (temperature_ph['year'] <= to_year)]\
    .groupby(temperature_ph['year'])[['year', 'temperature_celsius']]\
    .transform('mean')\
    .drop_duplicates()

rfph_mean_per_year = rainfall_ph[(rainfall_ph['year'] >= from_year) & (rainfall_ph['year'] <= to_year)]\
    .groupby(rainfall_ph['year'])[['year', 'rainfall_mm']]\
    .transform('mean')\
    .drop_duplicates()

ph_data = pd.DataFrame(tmph_mean_per_year)
ph_data['rainfall_mm'] = rfph_mean_per_year['rainfall_mm']
ph_data = ph_data.reset_index()\
    .drop(columns='index')\
    .merge(consumption_emissions[['Year', 'Philippines']], how='left', left_on='year', right_on='Year')\
    .drop(axis=0, columns=['Year'])\
    .rename(columns={'Philippines': 'consumption_emissions'})\
    .merge(territorial_emissions[['Year', 'Philippines']], how='left', left_on='year', right_on='Year')\
    .drop(axis=0, columns=['Year'])\
    .rename(columns={'Philippines': 'territorial_emissions'})\
    .merge(wb_data, how='left', left_on='year', right_on='Year')\
    .drop(axis=0, columns=['Year'])\
    .rename(columns={'SP.POP.TOTL':'population'})

In [ ]:
# temp_c, CO2 emissions (kt)
plt.scatter(ph_data['temperature_celsius'], ph_data['year'], s=ph_data['EN.ATM.CO2E.KT'].transform(math.log) * 2)

In [ ]:
plt.plot(ph_data['year'], ph_data['temperature_celsius'])

In [ ]:
sns.distplot(ph_data['temperature_celsius'][ph_data['year'] >= 1910])

In [ ]:
sns.lineplot(ph_data['EN.ATM.CO2E.PC'], ph_data['year'])